In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import model_from_yaml

os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
path = "C://Users//cs623//Docs//GitHub//cat-recognition//img"
path_dog = os.path.join(path, 'cat')
path_nodog = os.path.join(path, 'no_cat')
class_names = ['cat', 'no_cat']

In [3]:
total_img_num = 0
split = 0.2

for a in os.listdir(path_dog):
    total_img_num += len(os.listdir(os.path.join(path_dog, a)))

for a in os.listdir(path_nodog):
    total_img_num += len(os.listdir(os.path.join(path_nodog, a)))

total_val = total_img_num * split
total_train = total_img_num - total_val

print('total images:', total_img_num)
print('number of training data:', total_train)
print('number of validation:', total_val)

total images: 31911
number of training data: 25528.8
number of validation: 6382.200000000001


In [4]:
epochs = 26
IMG_HEIGHT = 112
IMG_WIDTH = 112
batch_size = 256

In [5]:
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,rotation_range=40, # Angle, 0-180
    width_shift_range=0.2, # horizontal shifting
    height_shift_range=0.2, # vertical shifting
    shear_range=0.2, # Shearing
    zoom_range=0.2, # Zooming
    horizontal_flip=True, # Flipping
    validation_split=split
)

train_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='training'
) # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='validation'
) # set as validation data

Found 25529 images belonging to 2 classes.
Found 6382 images belonging to 2 classes.


In [7]:
# all labels
labels = train_generator.classes

In [8]:
## check encoded label
labels = (train_generator.class_indices)
print(labels)

{'cat': 0, 'no_cat': 1}


In [9]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.8),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
with tf.device('/gpu:0'):
    history = model.fit_generator(
        train_generator,
        steps_per_epoch = total_train // batch_size,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = total_val // batch_size 
    )

Epoch 1/26
13/99 [==>...........................] - ETA: 2:24 - loss: 0.8294 - acc: 0.4958C:\Users\cs623\Anaconda3\envs\py36\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
99/99 [==============================] - 190s 2s/step - loss: 0.6745 - acc: 0.6015 - val_loss: 0.6946 - val_acc: 0.5454
Epoch 2/26
99/99 [==============================] - 186s 2s/step - loss: 0.6107 - acc: 0.6780 - val_loss: 0.7009 - val_acc: 0.5498
Epoch 3/26
99/99 [==============================] - 187s 2s/step - loss: 0.5935 - acc: 0.6917 - val_loss: 0.6837 - val_acc: 0.6047
Epoch 4/26
99/99 [==============================] - 189s 2s/step - loss: 0.5824 - acc: 0.6971 - val_loss: 0.7232 - val_acc: 0.5648
Epoch 5/26
14/99 [===>..........................] - ETA: 1:42 - loss: 0.5865 - acc: 0.6953

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='lower left')
plt.title('Training and Validation Loss')

plt.show()


## Save

In [ ]:
# serialize model to YAML
model.save("Detection.h5")

# serialize weights to HDF5
model.save_weights("Detection_weights.h5")